### Import Required Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from datetime import datetime
        
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from lightgbm import LGBMRegressor
from sklearn import metrics,model_selection
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error,mean_squared_error
from scipy.stats import skew
import optuna
from xgboost import XGBRegressor
from tqdm import tqdm
sampler = TPESampler(seed=10)
from sklearn.svm import SVR

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
import warnings
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
from scipy.special import boxcox1p
from scipy.stats import skew, boxcox_normmax, norm


from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV, TweedieRegressor,LinearRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor 

In [ ]:
BASE_PATH = '../input/tabular-playground-series-jan-2021'

### Reading the required files

In [ ]:
# Reading CSV
train_df = pd.read_csv(os.path.join(BASE_PATH,'train.csv'))
test_df = pd.read_csv(os.path.join(BASE_PATH,'test.csv'))
sample_df = pd.read_csv(os.path.join(BASE_PATH,'sample_submission.csv'))

### Performing EDA on the data

In [ ]:
# Checking Shapes
train_df.shape,test_df.shape,sample_df.shape

In [ ]:
# Lets Check for Missing Values
def missing_percentage(df):
    
    """A function for returning missing ratios."""
    
    total = df.isnull().sum().sort_values(
        ascending=False)[df.isnull().sum().sort_values(ascending=False) != 0]
    percent = (df.isnull().sum().sort_values(ascending=False) / len(df) *
               100)[(df.isnull().sum().sort_values(ascending=False) / len(df) *
                     100) != 0]
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
missing = missing_percentage(train_df)
if (len(missing)>0):
    fig, ax = plt.subplots(figsize=(20, 5))
    sns.barplot(x=missing.index, y='Percent', data=missing, palette='Reds_r')
    plt.xticks(rotation=90)
    display(missing.T.style.background_gradient(cmap='Reds', axis=1))
else :
    pass

In [ ]:
# Checking Sample Data
train_df.head()

In [ ]:
# Lets describe the dataframe and check for different statistics
train_df.describe().T

In [ ]:
# Lets Check for outliers and remove if any
i=1
plt.figure(figsize=(20,10))
for col in train_df.iloc[:,1:15]:
    plt.subplot(4,4,i)
    sns.boxplot(y = train_df[col],color='Blue')
    i+=1

    

Here we can note that cont7 and cont9 is having some amount of outliers

In [ ]:
train_df.shape

In [ ]:
# Removing Outliers
Q1 = train_df.cont7.quantile(0.25)
Q3 = train_df.cont7.quantile(0.75)
IQR = Q3 - Q1
train_df = train_df[(train_df.cont7 >= Q1 - 1.5*IQR) & (train_df.cont7 <= Q3 + 1.5*IQR)]

Q1 = train_df.cont9.quantile(0.25)
Q3 = train_df.cont9.quantile(0.75)
IQR = Q3 - Q1
train_df = train_df[(train_df.cont9 >= Q1 - 1.5*IQR) & (train_df.cont9 <= Q3 + 1.5*IQR)]

In [ ]:
train_df.shape

In [ ]:
# Lets check the multicollinearity
sns.set(font_scale=1.1)
correlation_train = train_df.iloc[:,1:15].corr()
mask = np.triu(correlation_train.corr())
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_train,
            annot=True,
            fmt='.1f',
            cmap='coolwarm',
            square=True,
            mask=mask,
            linewidths=1,
            cbar=False)

plt.show()

In [ ]:
# Lets create scatter plot to check corellation with target

i=1
plt.figure(figsize=(20,10))
for col in train_df.iloc[:,1:15]:
    plt.subplot(4,4,i)
    sns.scatterplot(train_df[col],train_df.target)
    i+=1


As such there is no correlation present between independent variable and target

Numerical Data plotting

In [ ]:
def srt_reg(y, df):
    fig, axes = plt.subplots(4, 4, figsize=(25, 80))
    axes = axes.flatten()

    for i, j in zip(df.select_dtypes(include=['number']).columns, axes):

        sns.regplot(x=i,
                    y=y,
                    data=df,
                    ax=j,
                    order=3,
                    ci=None,
                    color='#e74c3c',
                    line_kws={'color': 'black'},
                    scatter_kws={'alpha':0.4})
        j.tick_params(labelrotation=45)
        j.yaxis.set_major_locator(MaxNLocator(nbins=10))

        plt.tight_layout()

In [ ]:
srt_reg('target',train_df.iloc[:,1:16])

In [ ]:
skewed = ['cont3', 'cont4', 'cont5', 'cont6',
       'cont8', 'cont11', 'cont12', 'cont13', 'cont14',
       'target']

In [ ]:
# Finding skewness of the numerical features.
skew_features = np.abs(train_df[skewed].apply(lambda x: skew(x)).sort_values(
    ascending=False))

# Filtering skewed features.

high_skew = skew_features[skew_features > 0.3]

# Taking indexes of high skew.

skew_index = high_skew.index

# Applying boxcox transformation to fix skewness.

for i in skew_index:
    train_df[i] = boxcox1p(train_df[i], boxcox_normmax(train_df[i] + 1))
    
for i in skew_index:
    test_df[i] = boxcox1p(test_df[i], boxcox_normmax(test_df[i] + 1))

In [ ]:
# Split the dataset
y = train_df.target
X = train_df.iloc[:,1:15]

In [ ]:
kf = KFold(10, random_state=42)

In [ ]:
# Some parameters for ridge, lasso and elasticnet.
alphas_alt = [15.5, 15.6, 15.7, 15.8, 15.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [    0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

# ridge_cv

ridge = RidgeCV(alphas=alphas_alt,cv=kf,)

# lasso_cv:

# lasso = LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kf)

# elasticnet_cv:

# elasticnet = ElasticNetCV(max_iter=1e7,
#                  alphas=e_alphas,
#                  cv=kf,
#                  random_state=42,
#                  l1_ratio=e_l1ratio)

# svr:

# svr = SVR(C=21, epsilon=0.0099, gamma=0.00017, tol=0.000121)

# gradientboosting:

gbr = GradientBoostingRegressor(n_estimators=2900,
                                learning_rate=0.0161,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=17,
                                loss='huber',
                                random_state=42)

# lightgbm:

lightgbm = LGBMRegressor(objective='regression',
                         n_estimators=3500,
                         num_leaves=5,
                         learning_rate=0.00721,
                         max_bin=163,
                         bagging_fraction=0.35711,
                         n_jobs=-1,
                         bagging_seed=42,
                         feature_fraction_seed=42,
                         bagging_freq=7,
                         feature_fraction=0.1294,
                         min_data_in_leaf=8)

# xgboost:

xgboost = XGBRegressor(
    learning_rate=0.0139,
    n_estimators=4500,
    max_depth=4,
    min_child_weight=0,
    subsample=0.7968,
    colsample_bytree=0.4064,
    nthread=-1,
    scale_pos_weight=2,
    seed=42,
)


# hist gradient boosting regressor:

# hgrd= HistGradientBoostingRegressor( 
#     loss= 'least_squares',
#     max_depth= 2,
#     min_samples_leaf= 40,
#     max_leaf_nodes= 29,
#     learning_rate= 0.15,
#     max_iter= 225,
#     random_state=42
# )

# tweedie regressor:
 
# tweed = TweedieRegressor(alpha=0.005)


# stacking regressor:

# stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr,
#                                             xgboost, lightgbm,hgrd, tweed),
#                                 meta_regressor=xgboost,
#                                 use_features_in_secondary=True)

Cross Validation

In [ ]:
def model_check(X, y, estimators, cv):    
    model_table = pd.DataFrame()

    row_index = 0
    for est, label in zip(estimators, labels):

        MLA_name = label
        model_table.loc[row_index, 'Model Name'] = MLA_name

        cv_results = cross_validate(est,
                                    X,
                                    y,
                                    cv=cv,
                                    scoring='neg_root_mean_squared_error',
                                    return_train_score=True,
                                    n_jobs=-1)

        model_table.loc[row_index, 'Train RMSE'] = -cv_results[
            'train_score'].mean()
        model_table.loc[row_index, 'Test RMSE'] = -cv_results[
            'test_score'].mean()
        model_table.loc[row_index, 'Test Std'] = cv_results['test_score'].std()
        model_table.loc[row_index, 'Time'] = cv_results['fit_time'].mean()

        row_index += 1

    model_table.sort_values(by=['Test RMSE'],
                            ascending=True,
                            inplace=True)

    return model_table

In [ ]:
# Setting list of estimators and labels for them:

# estimators = [ridge, lasso, elasticnet, gbr, xgboost, lightgbm, svr, hgrd, tweed]
# labels = [
#     'Ridge', 'Lasso', 'Elasticnet', 'GradientBoostingRegressor',
#     'XGBRegressor', 'LGBMRegressor', 'SVR', 'HistGradientBoostingRegressor','TweedieRegressor'
# ]

estimators = [ridge, gbr, xgboost, lightgbm]

labels = ['Ridge',  'GradientBoostingRegressor','XGBRegressor', 'LGBMRegressor']

# Executing cross validation.

raw_models = model_check(X, y, estimators, kf)
display(raw_models.style.background_gradient(cmap='summer_r'))

In [ ]:
# Fitting the models on train data.

print('=' * 20, 'START Fitting', '=' * 20)
print('=' * 55)

# print(datetime.now(), 'StackingCVRegressor')
# stack_gen_model = stack_gen.fit(X.values, y.values)
# print(datetime.now(), 'Elasticnet')
# elastic_model_full_data = elasticnet.fit(X, y)
# print(datetime.now(), 'Lasso')
# lasso_model_full_data = lasso.fit(X, y)
# print(datetime.now(), 'Ridge')
# ridge_model_full_data = ridge.fit(X, y)
# print(datetime.now(), 'SVR')
# svr_model_full_data = svr.fit(X, y)
print(datetime.now(), 'GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)
# print(datetime.now(), 'XGboost')
# xgb_model_full_data = xgboost.fit(X, y)
# print(datetime.now(), 'Lightgbm')
# lgb_model_full_data = lightgbm.fit(X, y)
# print(datetime.now(), 'Hist')
# hist_full_data = hgrd.fit(X, y)
# print(datetime.now(), 'Tweed')
# tweed_full_data = tweed.fit(X, y)
print('=' * 20, 'FINISHED Fitting', '=' * 20)
print('=' * 58)

In [ ]:
# Blending models by assigning weights:
'''
def blend_models_predict(X):
    return (
#             (0.1 * elastic_model_full_data.predict(X)) +
#             (0.1 * lasso_model_full_data.predict(X)) +
#             (0.1 * ridge_model_full_data.predict(X)) +
#             (0.1 * svr_model_full_data.predict(X)) +
            (0.05 * gbr_model_full_data.predict(X)) +
            (0.1 * xgb_model_full_data.predict(X)) +
            (0.05 * lgb_model_full_data.predict(X)) 
#             (0.05 * hist_full_data.predict(X)) +
#             (0.1 * tweed_full_data.predict(X)) +
#             (0.25 * stack_gen_model.predict(X.values))
            )
'''

In [ ]:
submission = test_df
submission = submission.drop('id',axis=1)

In [ ]:
# sample_df.iloc[:,1:] = np.floor(np.expm1(blend_models_predict(submission)))
sample_df.iloc[:,1:] = (gbr_model_full_data.predict(submission)

# Defining outlier quartile ranges
# q1 = sample_df['target'].quantile(0.0050)
# q2 = sample_df['target'].quantile(0.99)

# Applying weights to outlier ranges to smooth them
# sample_df['target'] = sample_df['target'].apply(lambda x: x if x > q1 else x * 0.77)
# sample_df['target'] = sample_df['target'].apply(lambda x: x if x < q2 else x * 1.1)

In [ ]:
sample_df.head()

In [ ]:
sample_df.to_csv('submission.csv', index=False)
print(
    'Save submission',
    datetime.now(),
)